### **DEMO 2A**

In [ ]:
from docplex.cp.model import CpoModel

mtx = [[ 0,  0,  0,  5,  5,  5],
       [ 5,  0,  5, 10, 10, 10],
       [15, 10,  0,  5,  5, 10],
       [25, 25, 15,  0, 10,  5],
       [30, 30, 15, 15,  0, 10],
       [40, 40, 30, 30, 15,  0]]

m = CpoModel()

matriz = m.transition_matrix(mtx)

lotes = 3
pintura = []
ensamble = []
empaque = []

for l in range(lotes):
    pintura.append(m.interval_var(size=8*100, name='PINTURA_'+str(l)))
    ensamble.append(m.interval_var(size=12*100, name='ENSAMBLE_'+str(l)))
    empaque.append(m.interval_var(size=6*100, name='EMPAQUE_'+str(l)))
    m.add(m.end_before_start(pintura[l], ensamble[l], 6*60))
    m.add(m.end_before_start(ensamble[l], empaque[l]))

pre = m.interval_var(size=30, name='PRE')
pos = m.interval_var(size=35, name='POS')

# secuencia

s_temp = [pintura[l] for l in range(lotes)]
s_temp.append(pre)
s_temp.append(pos)  

seq = m.sequence_var(s_temp, name='SECUENCIA_PINTURA', types= [0, 5, 3, 4, 0])
m.add(seq)

m.add(m.no_overlap(seq, distance_matrix=matriz))
m.add(m.first(seq, pre))
m.add(m.last(seq, pos))

# funcional

#m.add(m.minimize(m.sum([m.end_of(empaque[l]) for l in range(lotes)])))

s = m.solve()
s.print_solution()



# gantt

import docplex.cp.utils_visu as visu
from pylab import rcParams
rcParams['figure.figsize'] = 20, 4

if s:
    visu.panel()
    for i in range(lotes):
        visu.interval( s.get_var_solution('PINTURA_'+str(i)) , 1 , 'PINTURA_'+str(i))
        visu.interval( s.get_var_solution('ENSAMBLE_'+str(i)) , 2 , 'ENSAMBLE_'+str(i))
        visu.interval( s.get_var_solution('EMPAQUE_'+str(i)) , 3 , 'EMPAQUE_'+str(i))
    visu.interval( s.get_var_solution('PRE') , 4 , 'PRE')
    visu.interval( s.get_var_solution('POS') , 5 , 'POS')
    visu.show(origin=0, horizon=30)


### **DEMO 2B** 

In [ ]:
from docplex.cp.model import CpoModel, transition_matrix
import docplex.cp.utils_visu as visu

m = CpoModel()

matriz = [[0, 12, 5],
          [12, 0 ,10],
          [5, 10, 0]]

ms = transition_matrix(matriz)

sf = m.state_function(trmtx=ms, name='sf')

p1 = m.interval_var(size=8,  name='P1')
p2 = m.interval_var(size=10, name='P2')
p3 = m.interval_var(size=6,  name='P3')


m.add(m.always_equal(sf, p1, 0))
m.add(m.always_equal(sf, p2, 1))
m.add(m.always_equal(sf, p3, 2))

m.add( m.minimize(   m.max_of([m.end_of(p1), m.end_of(p2), m.end_of(p3)] )))

s2 = m.solve()

s2.print_solution()

# 2 0 1

if s2:
    visu.panel()
    visu.interval(s2.get_var_solution(p1), 'pink', 'P1[0]')
    visu.interval(s2.get_var_solution(p2), 'lightblue', 'P2[1]')
    visu.interval(s2.get_var_solution(p3), 'lightgreen', 'P3[2]')

    # Gráfico de utilización de recursos
    #visu.sequence(name=sf.get_name(), segments=s2.get_var_solution(sf))
    visu.panel(name="función de estados")

    ff = s2.get_var_solution(sf)
    a = []
    for i in range( 1, len(ff.get_value())-1):
        if ff.get_value()[i][2]==-1:
            a.append((ff.get_value()[i][0], ff.get_value()[i][1], ant, ff.get_value()[i+1][2]))
        else:    
            a.append((ff.get_value()[i][0], ff.get_value()[i][1],ff.get_value()[i][2]))
            ant = ff.get_value()[i][2]


    #visu.function(segments=s2.get_var_solution(sf), style='segment', origin=0, horizon=30, color='khaki')     
    visu.function(segments=a, style='segment', origin=0, horizon=40, color='red')     


    visu.panel(name="Matriz de tiempos")
    visu.matrix(name="M1", matrix=matriz)
    


    visu.show(origin=0, horizon=30)
